In [1]:
import numpy as np
import xarray as xr
from tqdm import tqdm
from xcape.core import calc_cape
import os

In [2]:
itime=0

In [3]:
ds_tas = xr.open_dataset("/scratch/nf33/hk25_LSP/UK/MC/tas_3hourly.nc").isel(time=itime)
ta_surf = ds_tas["tas"].stack(ngrid=('latitude','longitude')).values.flatten()-273.15
ds_tdews = xr.open_dataset("/scratch/nf33/hk25_LSP/UK/MC/tdews_3hourly.nc").isel(time=itime)
tdew_surf = ds_tdews["tdews"].stack(ngrid=('latitude','longitude')).values.flatten()-273.15

In [4]:
ds_ta = xr.open_dataset("/scratch/nf33/hk25_LSP/UK/MC/ta_3hourly.nc").isel(time=itime)
ds_tdew = xr.open_dataset("/scratch/nf33/hk25_LSP/UK/MC/tdew_3hourly.nc").isel(time=itime)


In [6]:
ta_da = ds_ta["ta"].stack(ngrid=('latitude','longitude')).isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15
tdew_da = ds_tdew["tdew"].stack(ngrid=('latitude','longitude')).isel(pressure=slice(None, None, -1)).transpose('ngrid','pressure')-273.15

In [7]:
ta_val = ta_da.load().values
td_val = tdew_da.load().values
psurf = np.zeros(ta_val.shape[0])
psurf[:] = 1031

In [8]:
plvl = ds_ta.pressure.isel(pressure=slice(None, None, -1)).values.astype(float)

In [9]:
plvl

array([1000.,  975.,  950.,  925.,  900.,  875.,  850.,  800.,  750.,
        700.,  600.,  500.,  400.,  300.,  250.,  200.,  150.,  100.])

In [10]:
ta_val.shape, td_val.shape, ta_surf.shape

((646, 18), (646, 18), (646,))

In [11]:
%%time
cape, cin, mulev, zmulev = calc_cape(plvl, ta_val, td_val, 
                                         psurf,ta_surf, 
                                         tdew_surf,
                                         source ='most-unstable', vertical_lev="pressure")

CPU times: user 229 ms, sys: 1.85 ms, total: 231 ms
Wall time: 229 ms


In [13]:
cape_2d = cape.reshape(ds_ta.latitude.shape[0],ds_ta.longitude.shape[0])
cin_2d = cin.reshape(ds_tdew.latitude.shape[0],ds_tdew.longitude.shape[0])

In [14]:
cape_2d.shape

(19, 34)

In [14]:
out = xr.Dataset(
        {
            "cape": (("time","latitude","longitude"), cape_2d),
            "cin":  (("time","latitude","longitude"), cin_2d),
        },
        coords={
            "latitude": ds_ta.latitude,
            "longitude": ds_ta.longitude,
            "time":      ds_ta.time,
        },
    )
out.to_netcdf(f"/scratch/nf33/hk25_LSP/UK/MC/cape_cin_3hourly.nc")

In [18]:
ds_cape_cin = xr.open_mfdataset("/scratch/nf33/hk25_LSP/UK/MC/cape_cin/*.nc")

In [19]:
ds_cape_cin.to_netcdf("/scratch/nf33/hk25_LSP/UK/MC/cape_cin_3hourly.nc")

/jobfs/140911377.gadi-pbs/ipykernel_252349/4141139881.py:1: UserWarning: Times can't be serialized faithfully to int64 with requested units 'days since 2020-03-01'. Serializing with units 'hours since 2020-03-01' instead. Set encoding['dtype'] to floating point dtype to serialize with units 'days since 2020-03-01'. Set encoding['units'] to 'hours since 2020-03-01' to silence this warning .
  ds_cape_cin.to_netcdf("/scratch/nf33/hk25_LSP/UK/MC/cape_cin_3hourly.nc")
